# **Pokémon Diffusion<a id="top"></a>**

> #### `02-Diffusion-Model-Architecture.ipynb`

<i><small>**Alumno:** Alejandro Pequeño Lizcano<br>Última actualización: 12/06/2024</small></i></div>


Este notebook tiene como objetivo la construcción de la arquitectura del modelo de difusión que se encargará de predecir el ruido en cada paso de difusión.

Para ello, este notebook se dividirá en las siguientes secciones:

- [0. Imports](#0-imports)
- [1. Arquitectura del Modelo de Difusión](#1-arquitectura-del-modelo-de-difusión)

><span style="color: red; font-size: 1.5em;">&#9888;</span> **NOTA:** Antes de comenzar con la construcción de la arquitectura del modelo de difusión, se explicará brevemente el motivo de la elección de la arquitectura usada para la construcción del modelo de difusión.
</small></i>

El modelo de difusión se compone de dos partes:

- El **modelo en sí**: la red neuronal que se encarga de aprender a predecir el ruido en cada paso de difusión
- El proceso de difusión: el proceso iterativo que se encarga de aplicar el bloque de difusión a la imagen original o a la imagen con ruido en cada paso de difusión. Este proceso se repite un número fijo de veces para generar una imagen nueva con el ruido añadido.

Este modelo usado para la predicción del ruido, aprenderá posteriormente a quitarlo de manera iterativa hasta producir una imagen nueva. Según numerosos papers, en teoría, se podría ser cualquier red neuronal, ya que para desarrollar un proceso generativo basado en difusión, no existe una arquitectura específica y depende del conjunto de datos con el que se entrene. No obstante, la más usada para la síntesis de imágenes y, por ende la que se usará en este proyecto es la arquitectura encoder-decoder **U-Net**, gracias a sus características de recuperación de la información manteniendo la dimensionalidad de la imagen que hace que sea una de las arquitecturas más usadas en problemas generación de imágenes.

Esta arquitectura simétrica, se caracteriza por tener una parte de codificación (encoder) y una parte de decodificación (decoder) que se conectan entre sí. Además, cada bloque de decodificación se conecta con el bloque de codificación correspondiente mediante una operación de concatenación también conocido como conexiones residuales, que permiten al modelo ser más profundo, y por tanto, aprender mejor las características de la imagen. Todo ello, evitando el problema de desvanecimiento del gradiente (vanishing gradient problem), pues a la hora de retropropagar el error, la información de las capas más profundas se mantiene, al proporcionar un camino directo a través de las capas.

Más adelante, se explicará con más detalle la arquitectura de la U-Net y cómo se ha implementado en el modelo de difusión.

# 0. Imports

Una vez introducido el objetivo de este notebook, se importan las librerías necesarias para el desarrollo del apartado.

---

In [1]:
# Import necessary libraries
# =====================================================================
import configparser
import tensorflow as tf
from src.model.build_model import *
from src.utils import CONFIG_PATH
from src.utils.config import parse_config

2024-06-12 21:30:10.934121: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 21:30:11.292146: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Hyperparameters
# =====================================================================
config = configparser.ConfigParser()
config.read(CONFIG_PATH)

hyperparameters = parse_config(config, "hyperparameters")
IMG_SIZE = hyperparameters["img_size"]
NUM_CLASSES = hyperparameters["num_classes"]

# 1. Arquitectura del modelo de difusión

En este apartado, se construirá el modelo de difusión que se encargará de aprender a predecir el ruido en cada paso de difusión. Para ello, como se ha comentado con anterioridad, se construirá una red neuronal basada en la arquitectura **U-Net** que se encargará de generar una imagen nueva con el ruido añadido.

Para construir el modelo, nos apoyaremos en la función `build_unet` que se encargará de construir la arquitectura de la red neuronal: los bloques de codificación y decodificación de la red añadiendo la información temporal y la condicionalidad a través de la etiqueta de la imagen a generar, en nuestro caso, el tipo del Pokémon a generar.

- Funciones y Clases Auxiliares de `build_unet`

  - `SinusoidalTimeEmbeddingLayer`: Esta clase se encarga de calcular los embeddings sinusoidales de los pasos de tiempo.
  - `SelfAttentionLayer`: Esta clase implementa el mecanismo de auto-atención para el tensor de entrada.
  - `input_block()`: Función auxiliar para procesar los tensores de entrada.
  - `encoder_block()`: Función para construir los bloques del codificador.
  - `bottleneck_block()`: Función para construir el bloque de cuello de botella.
  - `decoder_block()`: Función para construir los bloques del decodificador.
  - `process_block()`: Función para procesar los bloques de la imagen, etiqueta y tiempo.

Dentro del bloque de difusión, se aplican transformaciones a cada uno de los parámetros de entrada, lo que puede incluir capas densas, normalización y activación Sigmoid Linear Unit (SiLU), también conocidad como Swish. Estas transformaciones capturan las relaciones y dependencias entre los diferentes aspectos de la entrada (imagen y tiempo). Finalmente, se calcula la imagen nueva con el ruido añadido.

El proceso de difusión utiliza una arquitectura de tipo **U-Net** modificada con bloques de difusión que toman en cuenta la imagen, su etiqueta y el tensor tiempo. Posteriormente, se realizan operaciones de convolución y pooling para reducir la resolución de la imagen mientras se procesa la información temporal y condicional. Luego, se realiza un proceso de decodificación utilizando operaciones de upsampling y concatenación para generar una imagen de salida que tiene la misma resolución que la imagen de entrada. Entre la codificación y decodificación, se añade una capa **MLP** para procesar toda la información codificada y generar una imagen de salida. Finalmente, se devuelve la imagen de salida.

El motivo de la función de pérdida, se explicará en el siguiente notebook, pero se basa en la función de pérdida **MSE** que se encargará de calcular la diferencia entre la imagen original y la imagen generada. Esta función de pérdida se encargará de minimizar el error entre ambas imágenes.

<i><small>**Más información** sobre el porqué matemático de la función de pérdida, aunque ya explicado, se puede encontrar en el paper [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) y una explicación más clara en la página [Diffusion Model Clearly Explained!](https://medium.com/@steinsfu/diffusion-model-clearly-explained-cd331bd41166).

><span style="color: red; font-size: 1.5em;">&#9888;</span> **NOTA:** El proceso matemático para llegar a esta fórmula es muy complejo para explicarlo en un simple notebook. Sin embargo, en el informe del proyecto se explicará con más detalle.
</small></i>

---

In [3]:
# Create the model
# =====================================================================
model = build_unet(IMG_SIZE, NUM_CLASSES)

# Compile the model
# =====================================================================
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_fn = tf.keras.losses.MeanSquaredError()
model.compile(loss=loss_fn, optimizer=optimizer)

# Show the model summary
# =====================================================================
model.summary()

2024-06-12 21:30:13.257461: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-12 21:30:13.309617: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-12 21:30:13.309943: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ y_input             │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ t_input             │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │      4,864 │ y_input[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sinusoidal_time_em… │ (None, 256)       │          0 │ t_input[0][0]     │
│ (SinusoidalTimeEmb… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ group_normalization │ (None, 256)       │        512 │ dense[0][0]       │
│ (GroupNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ sinusoidal_time_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ x_input             │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256)       │          0 │ group_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ group_normalizatio… │ (None, 256)       │        512 │ dense_1[0][0]     │
│ (GroupNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 64, 64,    │      1,792 │ x_input[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │     16,448 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256)       │          0 │ group_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ group_normalizatio… │ (None, 64, 64,    │        128 │ conv2d[0][0]      │
│ (GroupNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 64)        │          0 │ dense_2[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │     16,448 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64, 64,    │          0 │ group_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 64)  │          0 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 64)        │          0 │ dense_3[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 31,904,451 (121.71 MB)

 Trainable params: 31,904,451 (121.71 MB)

 Non-trainable params: 0 (0.00 B)

[BACK TO TOP](#top)
